In [ ]:
import numpy as np

In [ ]:
input_shape = (1, 5, 5)
spatial_scalars = np.zeros([2, 5, 5])
spatial_scalars[0][0][0] = 1
spatial_scalars[0][2][1] = 1
spatial_scalars[0][2][3] = 0.67
spatial_scalars[1][0][0] = 1
spatial_scalars[1][2][1] = 0.67
spatial_scalars[1][2][3] = 1
print(spatial_scalars)
input = np.array([
    [[0.1, 0.2, 0.3, 0.4, 0.5],
     [0.6, 0.9, 0.6, 0.9, 0.6],
     [0.9, 0.0, 0.9, 0.0, 0.9],
     [0.6, 0.9, 0.6, 0.9, 0.6],
     [0.1, 0.2, 0.3, 0.4, 0.5]]
    ])
filters = np.random.random((2, 3, 3))

In [ ]:
print("spatial_scalars:\n", spatial_scalars)
print("input:\n", input)
print("convolutional filters:\n", filters)

In [ ]:
# Simply applies convolutional filter and returns the inner product
def filter_pass(domain, x, y, filter):
    acc = 0 # bias put here as a f(filter, x, y)
    offset = (filter.shape[1]//2) # assume conv filters are square
    for dx in range(-offset, offset + 1):
        for dy in range(-offset, offset + 1):
            acc += domain[0][(x + dx) % domain.shape[1]][(y + dy) % domain.shape[2]] * filter[dx + offset][dy + offset]
    return acc


## Spatially variant scalar convolution operation, wrap around, no bias
def svconv2d_filter(input, filter, spatial_scalars):
    ret = np.empty(shape=input.shape)
    for x in range(input.shape[1]):
        for y in range(input.shape[2]):
            # 0 here for now because we only are trying this out on 1 channel, will iterate over all channels when time comes.
            ret[0][x][y] = spatial_scalars[x][y] * filter_pass(input, x, y, filter)
    return ret

def svconv2d(input, filters, spatial_scalars):
    return np.array([svconv2d_filter(input, filter=filters[i], spatial_scalars=spatial_scalars[i]) for i in range(filters.shape[0])])

# after = svconv2d_filter(input, filters[0], spatial_scalars[0])
print('input:\n', input)
print('after:\n', svconv2d(input, filters, spatial_scalars))

In [ ]:
1.83395446/2.73724546

In [2]:
## Circular padding:
import torch
import torch.nn.functional as F
input = torch.ones((1, 2, 28, 28))
print(input.shape)
print(F.pad(input, (1,1,1,1),"circular").shape)

torch.Size([1, 2, 28, 28])
torch.Size([1, 2, 30, 30])


In [59]:
inpt = torch.rand(1, 2, 3, 3)
a = F.pad(inpt, (1, 1, 1, 1), mode='circular')
conv = torch.rand(1, 2, 3, 3)

spatial_scalars = torch.zeros(2, 3, 3)

print(F.conv2d(a, conv))

output = torch.zeros(inpt.shape)
for chan in range(a.shape[1]):
    intermed = F.conv2d(a[:,chan:chan+1,:,:], conv[:,chan:chan+1,:,:])
    print(intermed.shape)
    output[:,chan:chan+1,:,:] += intermed
print(output)
assert not (torch.isclose(output[:,0,:,:] + output[:,1,:,:], F.conv2d(a, conv)).__contains__(False))

tensor([[[[5.4677, 5.4688, 5.0508],
          [5.1264, 5.1185, 5.0882],
          [5.0289, 5.3090, 5.2771]]]])
torch.Size([1, 1, 3, 3])
torch.Size([1, 1, 3, 3])
tensor([[[[2.0618, 2.4289, 1.8683],
          [2.2202, 2.2237, 2.0290],
          [2.2045, 2.1236, 2.3801]],

         [[3.4059, 3.0399, 3.1825],
          [2.9062, 2.8948, 3.0592],
          [2.8244, 3.1853, 2.8970]]]])


In [8]:
x = torch.randint(0, 9, (1, 2, 2, 2))
print(x)
a = torch.zeros((1, 2, 2, 2))
a[0][0][1][0] = 1
a[0][1][1][0] = 3
print(a)
print((a * x).sum(dim=1))

tensor([[[[5, 7],
          [7, 6]],

         [[5, 2],
          [8, 4]]]])
tensor([[[[0., 0.],
          [1., 0.]],

         [[0., 0.],
          [3., 0.]]]])
tensor([[[ 0.,  0.],
         [31.,  0.]]])


In [92]:
(1, 1) + (2,)

(1, 1, 2)

In [1]:
from svconv import SVConv2d
import torch
import torch.nn.functional as F

input = torch.rand(1, 2, 3, 3)

convlayer = SVConv2d(in_channels=2, out_channels=2, kernel_size=3, spatial_scalar_hint=input.size(), stride=1, padding=(1,1), padding_mode='circular')
convlayer(input)

AttributeError: 'SVConv2d' object has no attribute 'spatial_scalars'